<a href="https://colab.research.google.com/github/gdmcdonald/Cutting-Stock-Problem/blob/main/solve_cutting_stock_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solve cutting problem

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [ ]:
!pip install ortools


Solves a problem in which various lengths of wood must be cut with minimal waste from longer lengths of wood. https://en.wikipedia.org/wiki/Cutting_stock_problem


In [4]:
from ortools.linear_solver import pywraplp


def create_data_model():
    """Create the data for the example."""
    data = {}
    # These are the lengths you want to cut
    weights = [910, 837.8, 773.2, 716.2, 666.8, 625, 590.8, 564.2, 545.2, 533.8, 530, 533.8, 545.2, 564.2, 590.8, 625, 666.8, 716.2, 773.2, 837.8, 910]
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    # These are the stock you have available
    data['bin_capacity'] = [2400, 2400, 2400, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800]
    data['bins'] = list(range(len(data['bin_capacity'])))
    return data



def main():
    data = create_data_model()

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        return

    # Variables
    # x[i, j] = 1 if item i is packed in bin j.
    x = {}
    for i in data['items']:
        for j in data['bins']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

    # y[j] = 1 if bin j is used.
    y = {}
    for j in data['bins']:
        y[j] = solver.IntVar(0, 1, 'y[%i]' % j)

    # Constraints
    # Each item must be in exactly one bin.
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['bins']) == 1)

    # The amount packed in each bin cannot exceed its capacity.
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['weights'][i] for i in data['items']) <= y[j] *
            data['bin_capacity'][j])

    # Objective: minimize the number of bins used.
    solver.Minimize(solver.Sum([y[j] for j in data['bins']]))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        num_bins = 0
        for j in data['bins']:
            if y[j].solution_value() == 1:
                bin_items = []
                bin_weight = 0
                for i in data['items']:
                    if x[i, j].solution_value() > 0:
                        bin_items.append(i)
                        bin_weight += data['weights'][i]
                if bin_items:
                    num_bins += 1
                    print('Bin number', j)
                    print('  Items packed:', bin_items)
                    print('  Total weight:', bin_weight)
                    print()
        print()
        print('Number of bins used:', num_bins)
        print('Time = ', solver.WallTime(), ' milliseconds')
    else:
        print('The problem does not have an optimal solution.')


if __name__ == '__main__':
    main()

Bin number 0
  Items packed: [3, 6, 11, 12]
  Total weight: 2386.0

Bin number 1
  Items packed: [2, 17, 20]
  Total weight: 2399.4

Bin number 2
  Items packed: [4, 8, 10, 15]
  Total weight: 2367.0

Bin number 5
  Items packed: [0, 18]
  Total weight: 1683.2

Bin number 8
  Items packed: [7, 13, 16]
  Total weight: 1795.2

Bin number 9
  Items packed: [5, 9, 14]
  Total weight: 1749.6

Bin number 11
  Items packed: [1, 19]
  Total weight: 1675.6


Number of bins used: 7
Time =  1399  milliseconds
